In [6]:
pip install transformers torch tensorflow numpy


In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import random
import string


In [18]:
# Load GPT-2 model and tokenizer from Hugging Face
model_name = 'gpt2'
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Function to generate text using GPT-2
def generate_text_gpt2(prompt, max_length=200):
    # Tokenize the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate text
    with torch.no_grad():
        output = gpt2_model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                                no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=1.0)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example user input for GPT-2
user_prompt = "The impact of Artificial Intelligence on healthcare"
generated_paragraph_gpt2 = generate_text_gpt2(user_prompt)
print("Generated GPT-2 Text: \n", generated_paragraph_gpt2)

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated GPT-2 Text: 
 The impact of Artificial Intelligence on healthcare is not clear.

The most recent study, published in the journal Nature, found that AI could be used to improve healthcare in a number of ways. The researchers found a significant increase in patient satisfaction with healthcare, and a decrease in hospital admissions. They also found an increase of patients' satisfaction in their healthcare provider's services. This is a positive sign for healthcare. However, the researchers also noted that the AI may not be able to provide the same level of care as a human. In other words, it may be a bit more expensive to treat patients. It is also possible that patients may have a lower level or level-of-care. For example, a patient may need to be hospitalized for a few days to get the right treatment. But, if the patient is in pain, they may also need a more intensive treatment, such as surgery. And, patients who are not in need of treatment may want to seek out a doctor who 

In [11]:
# Create a placeholder text corpus file
with open('text_corpus.txt', 'w', encoding='utf-8') as f:
    f.write("This is a placeholder text corpus for the LSTM model. It contains some sample text to allow the code to run without a FileNotFoundError. You should replace this with your actual text data.")

In [12]:
# Example: Loading a text corpus for LSTM-based generation
text = ""
with open('text_corpus.txt', 'r', encoding='utf-8') as file:
    text = file.read().lower().replace('\n', ' ').replace('\r', '')

# Get unique characters
chars = sorted(set(text))
char_indices = {char: index for index, char in enumerate(chars)}
indices_char = {index: char for index, char in enumerate(chars)}

# Create sequences of characters and their corresponding next character
sequence_length = 100
sequences = []
next_chars = []
for i in range(0, len(text) - sequence_length, 1):
    sequences.append(text[i:i + sequence_length])
    next_chars.append(text[i + sequence_length])

# Convert sequences into one-hot encoded format
X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [19]:
# Building the LSTM model for text generation
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(128))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(chars), activation='softmax'))

# Compile the model
lstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

# Train the LSTM model
lstm_model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 3.0460
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 505ms/step - loss: 3.0286
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 3.0122   
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 2.9908
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 780ms/step - loss: 2.9644
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step - loss: 2.9178
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 497ms/step - loss: 2.8734
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step - loss: 2.8192
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - loss: 2.8580
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step - loss: 2.8076
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step - loss: 2.7863
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step - loss: 2.8047
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step - loss: 2.7588
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 650ms/step - loss: 2.7653
Epoch 15/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step - loss: 2.8006
Epoch 16/20
1/1 ━━━━━━━━━━━━━━━━━━━━

In [20]:
# Function to generate text with the trained LSTM model
def generate_lstm_text(seed, length=200):
    generated_text = seed
    # Convert seed to lowercase to match the training data
    seed = seed.lower()
    for i in range(length):
        x_pred = np.zeros((1, len(seed), len(chars)))
        for t, char in enumerate(seed):
            # Check if the character is in char_indices before accessing
            if char in char_indices:
                x_pred[0, t, char_indices[char]] = 1
            else:
                # Optionally, handle characters not in the vocabulary, e.g., skip or replace
                pass # Skipping the character
        # Ensure x_pred is not all zeros before predicting
        if np.sum(x_pred) > 0:
            # Use lstm_model for prediction
            pred_index = np.argmax(lstm_model.predict(x_pred, verbose=0), axis=-1)[0]
            generated_text += indices_char[pred_index]
            # Only update seed if a character was added
            if char in char_indices:
                 seed = seed[1:] + indices_char[pred_index]
            else:
                 seed = seed[1:] # Just remove the first character if it was skipped
        else:
            # If x_pred is all zeros, break or handle appropriately
            break
    return generated_text

# Example seed for LSTM text generation
seed_text = "The role of artificial intelligence"
generated_lstm_text = generate_lstm_text(seed_text)
print("\nGenerated LSTM Text: \n", generated_lstm_text)


Generated LSTM Text: 
 The role of artificial intelligence                                                                                                                                                                                                        


In [21]:
def run_text_generation():
    choice = input("Choose a model for text generation (GPT-2 / LSTM): ").strip().lower()

    if choice == "gpt-2":
        user_prompt = input("Enter a prompt for GPT-2: ")
        print("\nGenerating text using GPT-2...")
        print(generate_text_gpt2(user_prompt))

    elif choice == "lstm":
        seed_text = input("Enter a seed text for LSTM: ")
        print("\nGenerating text using LSTM...")
        print(generate_lstm_text(seed_text))

    else:
        print("Invalid choice. Please select either 'GPT-2' or 'LSTM'.")

# Run the model based on user choice
run_text_generation()


Choose a model for text generation (GPT-2 / LSTM): LSTM
Enter a seed text for LSTM:  Artificial intelligence is

Generating text using LSTM...
 Artificial intelligence is                                                                                                                                                                                                        
